In [14]:
# to save data from Vedomosti RSS to xml file to the same directory as the script placed
import csv
import requests
import xml.etree.ElementTree as ET
import datetime
import time
import os


In [15]:
#date and time for file name creation
datetime = datetime.datetime.now().strftime('%d%m%Y%H%M%S') 
datetime

'30122022123441'

In [16]:
#saving xml file with data from Vedomosti RSS
response = requests.get("https://www.vedomosti.ru/rss/news")
with open(f'vedomosti{datetime}.xml', "w", encoding="ISO-8859-1") as f:
    f.write(response.text)

In [17]:
#to add data from xml file to filesdata table 
import glob
import psycopg2

#the latest file name 
list_of_files = glob.glob('*.xml') 
latest_file = max(list_of_files, key=os.path.getmtime)
print(latest_file)

vedomosti30122022123441.xml


In [18]:
#data from xml file
with open(latest_file, encoding='utf8') as f:
    contents = f.read()
    print(contents)

<?xml version="1.0" encoding="UTF-8"?>
<rss version="2.0">
  <channel>
    <title>"Ведомости". Ежедневная деловая газета</title>
    <link>https://www.vedomosti.ru</link>
    <description>"Ведомости". Новости, 30.12.2022</description>
    <image>
      <url>https://cdn.vdmsti.ru/assets/rss_logo.gif</url>
      <title>"Ведомости". Ежедневная деловая газета</title>
      <link>https://www.vedomosti.ru</link>
    </image>
<item>
  <title>Путин заявил о формировании справедливого миропорядка силами России и Китая</title>
  <link>https://www.vedomosti.ru/politics/news/2022/12/30/957771-putin-formirovanii-spravedlivogo-miroporyadka</link>
  <guid>https://www.vedomosti.ru/politics/news/2022/12/30/957771-putin-formirovanii-spravedlivogo-miroporyadka</guid>
  <pdalink>https://www.vedomosti.ru/politics/news/2022/12/30/957771-putin-formirovanii-spravedlivogo-miroporyadka</pdalink>
  <author></author>
  <category>Политика</category>
  <enclosure url="https://cdn.vdmsti.ru/image/2022/a4/qfdw6/norma

In [19]:
#insert data from xml file to filesdata table
try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="news1")
    cursor = connection.cursor()

    postgres_insert = """ INSERT INTO filesdata (filename, xmldata) VALUES (%s,%s)"""
    data_to_insert = (f'{latest_file}', f'{contents}')
    cursor.execute(postgres_insert, data_to_insert)

    connection.commit()
    count = cursor.rowcount
    print(count, "record inserted successfully")

except (Exception, psycopg2.Error) as error:
    print("failed to insert", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

1 record inserted successfully
PostgreSQL connection is closed


In [20]:
#insert categories

try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="news1")
    cursor = connection.cursor()

    postgres_insert = """ WITH n AS (SELECT DISTINCT text(UNNEST(xpath('//item/category/text()', xmldata))) AS category
FROM filesdata WHERE filename LIKE 'vedomosti%')
INSERT INTO categories (categoryname)
SELECT category FROM n
LEFT JOIN categories ON n.category=categories.categoryname
WHERE category_id IS NULL;"""

    cursor.execute(postgres_insert)
    
    connection.commit()
    count = cursor.rowcount
    print(count, "record inserted successfully")

except (Exception, psycopg2.Error) as error:
    print("failed to insert", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

0 record inserted successfully
PostgreSQL connection is closed


In [21]:
#insert news

try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="news1")
    cursor = connection.cursor()

    postgres_insert = f""" WITH q AS 
	(WITH w AS 
		(WITH n AS 
			(SELECT text(UNNEST(xpath('//item/title/text()', xmldata))) AS title,
			text(UNNEST(xpath('//item/link/text()', xmldata))) AS link,
			text(UNNEST(xpath('//item/guid/text()', xmldata))) AS guid,
			text(UNNEST(xpath('//item/pdalink/text()', xmldata))) AS pdalink,
			text(UNNEST(xpath('//item/author/text()', xmldata))) AS author,
			text(UNNEST(xpath('//item/category/text()', xmldata))) AS category,
			text(UNNEST(xpath('//item/enclosure/@url', xmldata))) AS enclosure,
			CAST(text(UNNEST(xpath('//item/pubDate/text()', xmldata))) AS timestamp) AS pubDate
			FROM filesdata
			WHERE filename='{latest_file}')
		SELECT n.*, c.category_id FROM n
		JOIN categories c ON n.category = c.categoryname)
	SELECT w.* FROM w --include unique rows
	LEFT JOIN news nn ON w.guid = nn.guid
	WHERE nn.guid is NULL)
INSERT INTO news ( title, link, guid, pubdate, description, author, enclosure, pdalink, source_id, file_id)
SELECT  q.title, q.link, q.guid, q.pubdate, NULL, q.author, q.enclosure, q.pdalink, '1',
(SELECT file_id from filesdata WHERE filename = '{latest_file}')
FROM q"""
  
    cursor.execute(postgres_insert)
    
    connection.commit()
    count = cursor.rowcount
    print(count, "record inserted successfully")

except (Exception, psycopg2.Error) as error:
    print("failed to insert", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

6 record inserted successfully
PostgreSQL connection is closed


In [22]:
#insert newscategories

try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="news1")
    cursor = connection.cursor()

    postgres_insert = f""" INSERT INTO newscategories (news_id, category_id)
WITH a AS (WITH o AS (WITH l AS 
			(SELECT text(UNNEST(xpath('//item/guid/text()', xmldata))) AS guid,
			text(UNNEST(xpath('//item/category/text()', xmldata))) AS category
			FROM filesdata
			WHERE filename='{latest_file}')
		SELECT n.news_id, n.guid, l.category FROM news n JOIN l ON n.guid = l.guid)
	SELECT o.*, c.category_id FROM o JOIN categories c ON o.category = c.categoryname)
SELECT a.news_id, a.category_id FROM a 
LEFT JOIN newscategories nc ON a.news_id = nc.news_id
WHERE nc.news_id IS NULL"""
   
    cursor.execute(postgres_insert)
    
    connection.commit()
    count = cursor.rowcount
    print(count, "record inserted successfully")

except (Exception, psycopg2.Error) as error:
    print("failed to insert", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

6 record inserted successfully
PostgreSQL connection is closed
